# 0.0 IMPORTS

In [1]:
import pandas as pd
import numpy as np
import inflection
import os
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings


from datetime import datetime, timedelta


warnings.filterwarnings('ignore')

## 0.1 Loading Data

In [2]:
path = 'C:/Users/edils/repos/leilao_fechamento/data/5_minutos/'

In [3]:
df_raw_list = []

for file in os.listdir(path):
    if file.endswith('.csv'):
        filename_path = os.path.join(path, file)
        df = pd.read_csv(filename_path, encoding='iso-8859-1', sep=';')
        df_raw_list.append(df)

In [4]:
df_raw = pd.concat(df_raw_list, axis=0)

In [5]:
df_raw.head()

,Ativo,Data,Hora,Abertura,Máximo,Mínimo,Fechamento,Volume,Quantidade
0,AALR3,16/06/2023,16:45:00,"23,16","23,24","23,14","23,24","1.013.115,00",43.6
1,AALR3,16/06/2023,16:40:00,"23,21","23,21","23,17","23,18","81.198,00",3.5
2,AALR3,16/06/2023,16:35:00,"23,21","23,22","23,21","23,22","44.107,00",1.9
3,AALR3,16/06/2023,16:30:00,"23,21","23,22","23,20","23,21","97.485,00",4.2
4,AALR3,16/06/2023,16:25:00,"23,21","23,24","23,21","23,21","137.035,00",5.9


In [6]:
df_raw.dtypes

Ativo         object
Data          object
Hora          object
Abertura      object
Máximo        object
Mínimo        object
Fechamento    object
Volume        object
Quantidade    object
dtype: object

## 0.2 Helper Functions

In [7]:
def data_types(df1):
    #abertura
    df1['abertura'] = df1['abertura'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
    df1['abertura'] = df1['abertura'].apply(lambda x: x.replace('.', '') if isinstance(x, str) else x)
    df1['abertura'] = df1['abertura'].astype(float)

    #maximo
    df1['maximo'] = df1['maximo'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
    df1['maximo'] = df1['maximo'].apply(lambda x: x.replace('.', '') if isinstance(x, str) else x)
    df1['maximo'] = df1['maximo'].astype(float)

    #minimo
    df1['minimo'] = df1['minimo'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
    df1['minimo'] = df1['minimo'].apply(lambda x: x.replace('.', '') if isinstance(x, str) else x)
    df1['minimo'] = df1['minimo'].astype(float)

    #fechamento
    df1['fechamento'] = df1['fechamento'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
    df1['fechamento'] = df1['fechamento'].apply(lambda x: x.replace('.', '') if isinstance(x, str) else x)
    df1['fechamento'] = df1['fechamento'].astype(float)

    #volume
    df1['volume'] = df1['volume'].apply(lambda x: x.replace(".", "").replace(",", "."))
    df1['volume'] = df1['volume'].astype(float)

    #quantidade
    df1['quantidade'] = df1['quantidade'].apply(lambda x: x.replace(".", "") if isinstance(x, str) else x)
    df1['quantidade'] = df1['quantidade'].apply(lambda x: float(x))

    #data
    df1['data'] = df1['data'].apply(lambda x: x.replace('/','-'))
    df1['data'] = df1['data'].apply(lambda x: datetime.strptime(x, "%d-%m-%Y"))
    df1['data'] = pd.to_datetime(df1['data'], format='%d-%m-%Y')
    df1['hora'] = df1['hora'].apply(lambda x: datetime.strptime(x, "%H:%M:%S"))
    
    return df1

def rename_columns(df1):
    cols_old = df1.columns
    cols_new = cols_old.map(lambda x: inflection.underscore(x))
    df1.columns = cols_new
    df1 = df1.rename(columns={'máximo': 'maximo', 'mínimo':'minimo'})
    
    return df1

def changing_time(df1):
    #Horário de verão 2023-03-13
    data_verao = pd.to_datetime('2023-03-13')
    df1.loc[df1['data'] >= data_verao, 'hora'] += pd.Timedelta(hours=1)
    hora_leilao = (df1['hora'].dt.hour >= 16) & (df1['hora'].dt.hour < 18) 
    df1 = df1.loc[hora_leilao,:]
    return df1

def clean_futuros(df_win):
    cols_old = df_win.columns
    cols_new = cols_old.map(lambda x: inflection.underscore(x))
    df_win.columns = cols_new
    df_win = df_win.rename(columns={'máximo': 'maximo', 'mínimo':'minimo'})

    df_win['abertura'] = df_win['abertura'].astype(float)

    #maximo

    df_win['maximo'] = df_win['maximo'].astype(float)

    #minimo

    df_win['minimo'] = df_win['minimo'].astype(float)

    #fechamento

    df_win['fechamento'] = df_win['fechamento'].astype(float)

    #volume
    df_win['volume'] = df_win['volume'].apply(lambda x: x.replace(".", "").replace(",", "."))
    df_win['volume'] = df_win['volume'].astype(float)

    #quantidade
    df_win['quantidade'] = df_win['quantidade'].apply(lambda x: x.replace(".", "") if isinstance(x, str) else x)
    df_win['quantidade'] = df_win['quantidade'].apply(lambda x: float(x))

    #data
    df_win['data'] = df_win['data'].apply(lambda x: x.replace('/','-'))
    df_win['data'] = df_win['data'].apply(lambda x: datetime.strptime(x, "%d-%m-%Y"))
    df_win['data'] = pd.to_datetime(df_win['data'], format='%d-%m-%Y')
    df_win['hora'] = df_win['hora'].apply(lambda x: datetime.strptime(x, "%H:%M:%S"))
    df_win['variacao'] = (df_win['fechamento'] - df_win['abertura']) / df_win['abertura']
    
    #Horário de verão 2023-03-13
    data_verao = pd.to_datetime('2023-03-13')
    df_win.loc[df_win['data'] >= data_verao, 'hora'] += pd.Timedelta(hours=1)
    
    hora_leilao = (df_win['hora'].dt.hour >= 16) & (df_win['hora'].dt.hour < 18) 
    df_win = df_win.loc[hora_leilao,:]
    
    df_win['variacao'] = (df_win['fechamento'] - df_win['abertura']) / df_win['abertura']
    
    return df_win

palette = ["#00BFFF", "#DC143C", "#FFA07A", "#FFD700", "#8B008B", "#7CFC00", "#FF69B4", "#00CED1", "#FF6347", "#9400D3"]
def jupyter_settings(palette):
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [12, 6]
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 50
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set(font_scale=2)
    sns.set_theme(palette=palette)
    warnings.simplefilter("ignore")
    # plt.rcParams.update({'font.size': 10})
    sns.set_style("white")

    
jupyter_settings(palette)
    

# 1.0 DATA DESCRIPTION

In [8]:
df1 = df_raw.copy()

In [9]:
df1

,Ativo,Data,Hora,Abertura,Máximo,Mínimo,Fechamento,Volume,Quantidade
0,AALR3,16/06/2023,16:45:00,"23,16","23,24","23,14","23,24","1.013.115,00",43.6
1,AALR3,16/06/2023,16:40:00,"23,21","23,21","23,17","23,18","81.198,00",3.5
2,AALR3,16/06/2023,16:35:00,"23,21","23,22","23,21","23,22","44.107,00",1.9
3,AALR3,16/06/2023,16:30:00,"23,21","23,22","23,20","23,21","97.485,00",4.2
4,AALR3,16/06/2023,16:25:00,"23,21","23,24","23,21","23,21","137.035,00",5.9
...,...,...,...,...,...,...,...,...,...
15466,ZAMP3,30/09/2022,10:20:00,"6,58","6,58","6,56","6,58","26.297,00",4.000
15467,ZAMP3,30/09/2022,10:15:00,"6,53","6,57","6,53","6,56","39.290,00",6.000
15468,ZAMP3,30/09/2022,10:10:00,"6,52","6,55","6,50","6,53","81.014,00",12.400
15469,ZAMP3,30/09/2022,10:05:00,"6,55","6,55","6,54","6,54","13.736,00",2.100


## 1.1 Rename Columns

In [10]:
df1 = rename_columns(df1)

## 1.2 Data Types

In [11]:
df1.dtypes

ativo         object
data          object
hora          object
abertura      object
maximo        object
minimo        object
fechamento    object
volume        object
quantidade    object
dtype: object

In [12]:
df1 = data_types(df1)

## 1.3 Check NA

In [13]:
df1.isna().sum()

ativo         0
data          0
hora          0
abertura      0
maximo        0
minimo        0
fechamento    0
volume        0
quantidade    0
dtype: int64

## 1.4 Data Shape

In [14]:
df1.shape

(4436242, 9)

## 1.5 Changing Time

In [15]:
df1 = changing_time(df1)

## 1.6 Create Response Variable

In [16]:
df1['variacao'] = (df1['fechamento'] - df1['abertura']) / df1['abertura']

In [17]:
df1.tail()

,ativo,data,hora,abertura,maximo,minimo,fechamento,volume,quantidade,variacao
15394,ZAMP3,2022-09-30,1900-01-01 16:20:00,683.0,685.0,681.0,685.0,139226.0,20400.0,0.002928
15395,ZAMP3,2022-09-30,1900-01-01 16:15:00,682.0,685.0,682.0,683.0,122991.0,18000.0,0.001466
15396,ZAMP3,2022-09-30,1900-01-01 16:10:00,683.0,684.0,681.0,682.0,150120.0,22000.0,-0.001464
15397,ZAMP3,2022-09-30,1900-01-01 16:05:00,684.0,685.0,682.0,683.0,54690.0,8000.0,-0.001462
15398,ZAMP3,2022-09-30,1900-01-01 16:00:00,681.0,685.0,679.0,685.0,228889.0,33600.0,0.005874


# 2.0 Feature Engineering

In [43]:
df2 = df1.copy()

In [44]:
dia_bruxa = '2023-06-16'
# #retirar dia da bruxa
# df2 = df2.loc[df2['data'] < '2023-06-16',:]

#retirar apenas o leilao do dia da bruxa
df2 = df2.loc[~((df2['data'] == dia_bruxa) & (df2['hora'].dt.hour == 17) & (df2['hora'].dt.minute == 45)),:]

# 3.0 SELECAO DE PAPEIS

## Selecionar os ultimos 30 dias uteis

In [45]:
ultimos_30_dias = df2['data'].unique()[:30]

In [46]:
df2_30 = df2.loc[df2['data'].isin(ultimos_30_dias),:]

## Candle Positivo/Negativo

In [47]:
df2_30['tipo_candle'] = np.zeros
df2_30['tipo_candle'] = df2_30.apply(lambda x: 'positivo' if x['variacao'] > 0 else ('zero' if x['variacao'] == 0 else 'negativo'), axis=1)

## Separar em primeira e segunda hora

In [48]:
df_primeira_hora = df2_30.loc[df2_30['hora'].dt.hour < 17,:]
df_segunda_hora = df2_30.loc[(df2_30['hora'].dt.hour >= 17) & (df2_30['hora'].dt.minute != 50),:]
df_leilao = df2_30.loc[(df2_30['hora'].dt.hour >= 17) & (df2_30['hora'].dt.minute == 50),:]

In [49]:
df_segunda_hora.head(30)

,ativo,data,hora,abertura,maximo,minimo,fechamento,volume,quantidade,variacao,tipo_candle
1,AALR3,2023-06-16,1900-01-01 17:40:00,2321.0,2321.0,2317.0,2318.0,81198.0,3.5,-0.001293,negativo
2,AALR3,2023-06-16,1900-01-01 17:35:00,2321.0,2322.0,2321.0,2322.0,44107.0,1.9,0.000431,positivo
3,AALR3,2023-06-16,1900-01-01 17:30:00,2321.0,2322.0,2320.0,2321.0,97485.0,4.2,0.000000,zero
4,AALR3,2023-06-16,1900-01-01 17:25:00,2321.0,2324.0,2321.0,2321.0,137035.0,5.9,0.000000,zero
5,AALR3,2023-06-16,1900-01-01 17:20:00,2321.0,2323.0,2321.0,2321.0,55720.0,2.4,0.000000,zero
6,AALR3,2023-06-16,1900-01-01 17:15:00,2321.0,2322.0,2321.0,2321.0,13927.0,600.0,0.000000,zero
7,AALR3,2023-06-16,1900-01-01 17:10:00,2321.0,2322.0,2320.0,2321.0,25529.0,1.1,0.000000,zero
8,AALR3,2023-06-16,1900-01-01 17:05:00,2321.0,2322.0,2320.0,2321.0,13926.0,600.0,0.000000,zero
9,AALR3,2023-06-16,1900-01-01 17:00:00,2323.0,2324.0,2321.0,2321.0,46448.0,2.0,-0.000861,negativo
73,AALR3,2023-06-15,1900-01-01 17:45:00,2320.0,2320.0,2316.0,2320.0,41715.0,1.8,0.000000,zero


## Separar 1,3,5,7,10,15,20,25,30 dias

In [50]:
#Separar primeira hora nesses dias
dias_lista = [1,3,5,7,10,15,20,25,30]
df_dias_p_hora = {}

for dias in dias_lista:
    ultimos_dias = df_primeira_hora['data'].unique()[:dias]
    df_filtrado = df_primeira_hora.loc[df_primeira_hora['data'].isin(ultimos_dias),:]
    df_dias_p_hora[dias] = df_filtrado.copy()
    
#Separar segunda hora
df_dias_s_hora = {}

for dias in dias_lista:
    ultimos_dias = df_segunda_hora['data'].unique()[:dias]
    df_filtrado = df_segunda_hora.loc[df_segunda_hora['data'].isin(ultimos_dias),:]
    df_dias_s_hora[dias] = df_filtrado.copy()

#Separar leilao
df_dias_leilao = {}

for dias in dias_lista:
    ultimos_dias = df_leilao['data'].unique()[:dias]
    df_filtrado = df_leilao.loc[df_leilao['data'].isin(ultimos_dias),:]
    df_dias_leilao[dias] = df_filtrado.copy()

## Média e Dias Positivos

In [52]:
#media e dias positivos para todos horarios da primeira hora
df_dias_fst_p_result = {}

for key, df in df_dias_p_hora.items():
    df_result = (df.loc[df['tipo_candle'] == 'positivo',['ativo','data','variacao','volume','tipo_candle']]
                          .groupby(['ativo', 'data'])
                          .agg(medias = ('variacao', 'mean'),
                               std = ('variacao', 'std'),
                               num_pos = ('tipo_candle', lambda x: (x == 'positivo').sum()),
                               med_volume = ('volume', 'mean')).reset_index())
    df_dias_fst_p_result[key] = df_result
    
#media e dias positivos para todos horarios da segunda hora
df_dias_snd_p_result = {}

for key, df in df_dias_s_hora.items():
    df_result = (df.loc[df['tipo_candle'] == 'positivo',['ativo','data','variacao','volume','tipo_candle']]
                          .groupby(['ativo', 'data'])
                          .agg(medias = ('variacao', 'mean'),
                               std = ('variacao', 'std'),
                               num_pos = ('tipo_candle', lambda x: (x == 'positivo').sum()),
                               med_volume = ('volume', 'mean')).reset_index())
    df_dias_snd_p_result[key] = df_result

#media e dias positivos para todos horarios do leilao
df_dias_leilao_p_result = {}

for key, df in df_dias_leilao.items():
    df_result = (df.loc[df['tipo_candle'] == 'positivo',['ativo','data','variacao','volume','tipo_candle']]
                          .groupby(['ativo', 'data'])
                          .agg(medias = ('variacao', 'mean'),
                               num_pos = ('tipo_candle', lambda x: (x == 'positivo').sum()),
                               med_volume = ('volume', 'mean')).reset_index())
    df_dias_leilao_p_result[key] = df_result    
    

In [54]:
df_dias_leilao_p_result[5]

,ativo,data,medias,num_pos,med_volume
0,AALR3,2023-06-09,0.001292,1,1174071.0
1,AALR3,2023-06-12,0.001717,1,1712405.0
2,AALR3,2023-06-13,0.002582,1,1623312.0
3,AALR3,2023-06-15,0.000862,1,2563278.0
4,ABCB4,2023-06-14,0.001119,1,1509369.0
...,...,...,...,...,...
617,YDUQ3,2023-06-14,0.005042,1,7834154.0
618,ZAMP3,2023-06-09,0.008457,1,596150.0
619,ZAMP3,2023-06-13,0.013158,1,501188.0
620,ZAMP3,2023-06-14,0.002105,1,314121.0


## Média e Dias Negativos

In [27]:
#media e dias positivos para todos horarios da primeira hora
df_dias_fst_n_result = {}

for key, df in df_dias_p_hora.items():
    df_result = (df.loc[df['tipo_candle'] == 'negativo',['ativo','data','variacao','volume','tipo_candle']]
                          .groupby(['ativo', 'data'])
                          .agg(medias = ('variacao', 'mean'),
                               std = ('variacao', 'std'),
                               num_neg = ('tipo_candle', lambda x: (x == 'negativo').sum()),
                               med_volume = ('volume', 'mean')).reset_index())
    df_dias_fst_n_result[key] = df_result
    
#media e dias positivos para todos horarios da segunda hora
df_dias_snd_n_result = {}

for key, df in df_dias_s_hora.items():
    df_result = (df.loc[df['tipo_candle'] == 'negativo',['ativo','data','variacao','volume','tipo_candle']]
                          .groupby(['ativo', 'data'])
                          .agg(medias = ('variacao', 'mean'),
                               std = ('variacao', 'std'),
                               num_neg = ('tipo_candle', lambda x: (x == 'negativo').sum()),
                               med_volume = ('volume', 'mean')).reset_index())
    df_dias_snd_n_result[key] = df_result

#media e dias positivos para todos horarios do leilao
df_dias_leilao_n_result = {}

for key, df in df_dias_leilao.items():
    df_result = (df.loc[df['tipo_candle'] == 'negativo',['ativo','data','variacao','volume','tipo_candle']]
                          .groupby(['ativo', 'data'])
                          .agg(medias = ('variacao', 'mean'),
                               num_neg = ('tipo_candle', lambda x: (x == 'negativo').sum()),
                               med_volume = ('volume', 'mean')).reset_index())
    df_dias_leilao_n_result[key] = df_result   

## DF final agrupado por hora e lado

In [55]:
#final para os dias positivos primeira hora
df_dias_fst_p_final = {}

for key, df in df_dias_fst_p_result.items():
    df_final = (df.groupby('ativo')
                            .agg(num_pos = ('num_pos', 'sum'),
                                 media = ('medias', 'mean'),
                                 std = ('std', 'mean'),
                                 med_volume = ('med_volume','mean'))                             
                            .reset_index())
    
    df_final['ranking'] = (df_final['media'])
    df_dias_fst_p_final[key] = df_final

#final para os dias positivos segunda hora
df_dias_snd_p_final = {}

for key, df in df_dias_snd_p_result.items():
    df_final = (df.groupby('ativo')
                            .agg(num_pos = ('num_pos', 'sum'),
                                 media = ('medias', 'mean'),
                                 std = ('std', 'mean'),
                                med_volume = ('med_volume','mean'))                             
                            .reset_index())
    df_final['ranking'] = (df_final['media'])
    df_dias_snd_p_final[key] = df_final

#final para os dias negativos primeira hora
df_dias_fst_n_final = {}

for key, df in df_dias_fst_n_result.items():
    df_final = (df.groupby('ativo')
                            .agg(num_neg = ('num_neg', 'sum'),
                                 media = ('medias', 'mean'),
                                 std = ('std', 'mean'),
                                med_volume = ('med_volume','mean'))                             
                            .reset_index())
    df_final['ranking'] = (df_final['media'])
    df_dias_fst_n_final[key] = df_final

#final para os dias negativos segunda hora
df_dias_snd_n_final = {}

for key, df in df_dias_snd_n_result.items():
    df_final = (df.groupby('ativo')
                            .agg(num_neg = ('num_neg', 'sum'),
                                 media = ('medias', 'mean'),
                                 std = ('std', 'mean'),
                                med_volume = ('med_volume','mean'))                             
                            .reset_index())
    
    df_final['ranking'] = (df_final['media'])
    df_dias_snd_n_final[key] = df_final

#final para os dias positivos leilao
df_dias_leilao_p_final = {}

for key, df in df_dias_leilao_p_result.items():
    df_final = (df.groupby('ativo')
                            .agg(num_pos = ('num_pos', 'sum'),
                                 media = ('medias', 'mean'),
                                 std = ('medias', 'std'),
                                med_volume = ('med_volume','mean'))                             
                            .reset_index())
    
    df_final['ranking'] = (df_final['media'])
    df_dias_leilao_p_final[key] = df_final
    
#final para os dias negativos leilao
df_dias_leilao_n_final = {}

for key, df in df_dias_leilao_n_result.items():
    df_final = (df.groupby('ativo')
                            .agg(num_neg = ('num_neg', 'sum'),
                                 media = ('medias', 'mean'),
                                  std = ('medias', 'std'),
                                med_volume = ('med_volume','mean'))                             
                            .reset_index())
    
    df_final['ranking'] = (df_final['media'])
    df_dias_leilao_n_final[key] = df_final


In [57]:
df_dias_leilao_n_final[30].sort_values(by='ranking')

,ativo,num_neg,media,std,med_volume,ranking
120,IGBR3,1,-0.045058,NaN,3.562100e+04,-0.045058
117,HOOT4,5,-0.032568,0.018634,5.035200e+03,-0.032568
161,MNPR3,1,-0.023109,NaN,1.230600e+04,-0.023109
266,VIVR3,9,-0.022448,0.009397,2.047927e+05,-0.022448
250,TRAD3,5,-0.019473,0.009939,4.230278e+05,-0.019473
...,...,...,...,...,...,...
230,SMAC11,6,-0.000912,0.000501,1.876458e+05,-0.000912
275,WSPFUT,13,-0.000899,0.000568,3.720943e+06,-0.000899
274,WINFUT,13,-0.000658,0.000496,2.561712e+09,-0.000658
72,DI1FUT,8,-0.000458,0.000180,3.410073e+08,-0.000458


## Percorrer todos buscando os melhores

In [67]:
#percorrer todos os positivos da primeira hora por ranking e somar
ranking_fst_p = {}
volume_fst_p = {}
std_fst_p = {}
volume = 100000

for keys, df in df_dias_fst_p_final.items():
    filtro_volume = df[df['med_volume'] >= volume]['ativo']
    df_filtrado_vol = df[df['ativo'].isin(filtro_volume)]
    
    for ativo, ranking in df_filtrado_vol.groupby('ativo')['ranking'].sum().items():
        if ativo in ranking_fst_p:
            ranking_fst_p[ativo] += ranking
            volume_fst_p[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_fst_p[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            
        else:
            ranking_fst_p[ativo] = ranking
            volume_fst_p[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_fst_p[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            

df_soma_ranking_fst_p = pd.DataFrame({'ativo':list(ranking_fst_p.keys()),
                                'soma_ranking': list(ranking_fst_p.values()),
                                'med_volume': list(volume_fst_p.values()),
                                'std': list(std_fst_p.values())}) 

#percorrer todos os positivos da segunda hora por ranking e somar
ranking_snd_p = {}
volume_snd_p = {}
std_snd_p = {}
volume = 100000

for keys, df in df_dias_snd_p_final.items():
    filtro_volume = df[df['med_volume'] >= volume]['ativo']
    df_filtrado_vol = df[df['ativo'].isin(filtro_volume)]
    
    for ativo, ranking in df_filtrado_vol.groupby('ativo')['ranking'].sum().items():
        if ativo in ranking_snd_p:
            ranking_snd_p[ativo] += ranking
            volume_snd_p[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_snd_p[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            
        else:
            ranking_snd_p[ativo] = ranking
            volume_snd_p[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_snd_p[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            
df_soma_ranking_snd_p = pd.DataFrame({'ativo':list(ranking_snd_p.keys()),
                                'soma_ranking': list(ranking_snd_p.values()),
                                'med_volume': list(volume_snd_p.values()),
                                'std': list(std_snd_p.values())})

#percorrer todos os negativos da primeira hora por ranking e somar
ranking_fst_n = {}
volume_fst_n = {}
std_fst_n = {}
volume = 100000

for keys, df in df_dias_fst_n_final.items():
    filtro_volume = df[df['med_volume'] >= volume]['ativo']
    df_filtrado_vol = df[df['ativo'].isin(filtro_volume)]
    
    for ativo, ranking in df_filtrado_vol.groupby('ativo')['ranking'].sum().items():
        if ativo in ranking_fst_n:
            ranking_fst_n[ativo] += ranking
            volume_fst_n[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_fst_n[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            
        else:
            ranking_fst_n[ativo] = ranking
            volume_fst_n[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_fst_n[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()

df_soma_ranking_fst_n = pd.DataFrame({'ativo':list(ranking_fst_n.keys()),
                                'soma_ranking': list(ranking_fst_n.values()),
                                'med_volume': list(volume_fst_n.values()),
                                'std': list(std_fst_n.values())})

#percorrer todos os negativos da segunda hora por ranking e somar
ranking_snd_n = {}
volume_snd_n = {}
std_snd_n = {}
volume = 100000

for keys, df in df_dias_snd_n_final.items():
    filtro_volume = df[df['med_volume'] >= volume]['ativo']
    df_filtrado_vol = df[df['ativo'].isin(filtro_volume)]
    
    for ativo, ranking in df_filtrado_vol.groupby('ativo')['ranking'].sum().items():
        if ativo in ranking_snd_n:
            ranking_snd_n[ativo] += ranking
            volume_snd_n[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_snd_n[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            
        else:
            ranking_snd_n[ativo] = ranking
            volume_snd_n[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_snd_n[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()

df_soma_ranking_snd_n = pd.DataFrame({'ativo':list(ranking_snd_n.keys()),
                                'soma_ranking': list(ranking_snd_n.values()),
                                'med_volume': list(volume_snd_n.values()),
                                'std': list(std_snd_n.values())})

#percorrer todos os positivos do leilao por ranking e somar
ranking_leilao_p = {}
volume_leilao_p = {}
std_leilao_p = {}
volume = 1000000

for keys, df in df_dias_leilao_p_final.items():
    filtro_volume = df[df['med_volume'] >= volume]['ativo']
    df_filtrado_vol = df[df['ativo'].isin(filtro_volume)]
    
    for ativo, ranking in df_filtrado_vol.groupby('ativo')['ranking'].sum().items():
        if ativo in ranking_leilao_p:
            ranking_leilao_p[ativo] += ranking
            volume_leilao_p[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_leilao_p[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            
        else:
            ranking_leilao_p[ativo] = ranking
            volume_leilao_p[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_leilao_p[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()

df_soma_ranking_leilao_p = pd.DataFrame({'ativo':list(ranking_leilao_p.keys()),
                                'soma_ranking': list(ranking_leilao_p.values()),
                                'med_volume': list(volume_leilao_p.values()),
                                'std': list(std_leilao_p.values())})

#percorrer todos os negativos do leilao por ranking e somar
ranking_leilao_n = {}
volume_leilao_n = {}
std_leilao_n = {}
volume = 1000000

for keys, df in df_dias_leilao_n_final.items():
    filtro_volume = df[df['med_volume'] >= volume]['ativo']
    df_filtrado_vol = df[df['ativo'].isin(filtro_volume)]
    
    for ativo, ranking in df_filtrado_vol.groupby('ativo')['ranking'].sum().items():
        if ativo in ranking_leilao_n:
            ranking_leilao_n[ativo] += ranking
            volume_leilao_n[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_leilao_n[ativo] += df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()
            
        else:
            ranking_leilao_n[ativo] = ranking
            volume_leilao_n[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['med_volume'].mean()
            std_leilao_n[ativo] = df_filtrado_vol[df_filtrado_vol['ativo'] == ativo]['std'].mean()

df_soma_ranking_leilao_n = pd.DataFrame({'ativo':list(ranking_leilao_n.keys()),
                                'soma_ranking': list(ranking_leilao_n.values()),
                                'med_volume': list(volume_leilao_n.values()),
                                'std': list(std_leilao_n.values())})


In [68]:
df_soma_ranking_leilao_p.sort_values(by='soma_ranking', ascending=False).head(10)

,ativo,soma_ranking,med_volume,std
22,IFCM3,0.108245,1.178428e+07,NaN
31,MODL3,0.067103,4.704127e+07,NaN
21,HBSA3,0.059392,1.673088e+07,NaN
29,MEGA3,0.058297,2.001033e+07,NaN
32,MRVE3,0.056267,1.124863e+08,NaN
19,HAPV3,0.054734,4.595134e+08,NaN
35,ONCO3,0.054675,1.820600e+07,NaN
24,INTB3,0.052967,5.300591e+07,NaN
156,ASAI3,0.051574,2.897423e+08,NaN
149,INBR32,0.048949,1.870768e+07,NaN


In [69]:
df_soma_ranking_snd_p.sort_values(by='soma_ranking', ascending=False).head(10)

,ativo,soma_ranking,med_volume,std
144,TRAD3,0.192688,2.386991e+07,0.293185
163,AMER3,0.073246,3.790516e+06,0.003721
130,SEQL3,0.064062,1.671765e+06,NaN
174,OIBR3,0.063492,1.458151e+06,0.009383
87,LIGT3,0.062444,8.827317e+06,0.051689
96,MLAS3,0.050222,1.221747e+06,0.030212
154,VIIA3,0.049022,2.280403e+07,0.022016
70,HBSA3,0.046430,3.709361e+06,0.011389
116,QUAL3,0.044208,4.499575e+06,0.026475
8,ANIM3,0.042376,4.678827e+06,0.020892


In [33]:
df_soma_ranking_snd_n.sort_values(by='soma_ranking', ascending=True).head(10)

,ativo,soma_ranking,med_volume
76,IFCM3,-0.085704,1.315759e+06
8,AMER3,-0.080760,1.568054e+06
136,SEQL3,-0.075336,1.856790e+06
99,MLAS3,-0.058709,1.784370e+06
160,VIIA3,-0.049739,1.816326e+07
182,AERI3,-0.045920,1.073378e+06
90,LIGT3,-0.045434,7.839500e+06
73,HAPV3,-0.044234,6.029528e+07
175,TRAD3,-0.042178,4.190134e+05
46,CVCB3,-0.041604,1.585280e+07


In [34]:
df_soma_ranking_leilao_n.sort_values(by='soma_ranking', ascending=True).head(10)

,ativo,soma_ranking,med_volume
82,TRAD3,-0.143713,6.845032e+06
6,AMER3,-0.111621,1.392177e+07
147,SEQL3,-0.057032,7.470073e+06
150,VIIA3,-0.051273,7.108940e+07
55,LIGT3,-0.047792,3.397076e+07
81,TEND3,-0.044868,1.792858e+07
19,CASH3,-0.044770,3.074913e+07
92,XPBR31,-0.044371,4.650169e+07
20,CBAV3,-0.041558,3.416937e+07
58,MILS3,-0.040242,1.290612e+07


## Juntando em um DF resultante

In [89]:
# Crie um dicionário com os 4 dataframes
dicionario_dataframes = {
    'dataframe1': df_soma_ranking_leilao_p,
    'dataframe2': df_soma_ranking_snd_p,
    'dataframe3': df_soma_ranking_snd_n,
    'dataframe4': df_soma_ranking_leilao_n,
}

# Crie um dataframe vazio para armazenar os resultados
df_resultante = pd.DataFrame()

# Percorra cada dataframe no dicionário
for nome_dataframe, dataframe in dicionario_dataframes.items():
    # Agrupe os dados por 'ativo' e some a coluna 'soma_ranking'
    soma_ranking = dataframe.groupby('ativo')['soma_ranking'].sum()
    
    # Agrupe os dados por 'ativo' e some a coluna 'std'
    std = dataframe.groupby('ativo')['std'].mean()
    
    # Agrupe os dados por 'ativo' e calcule a média da coluna 'med_volume'
    media_volume = dataframe.groupby('ativo')['med_volume'].mean()
    
    # Crie um dataframe temporário com as colunas 'ativo', 'soma_ranking' e 'med_volume'
    df_temporario = pd.DataFrame({'ativo': soma_ranking.index, 'soma_ranking': soma_ranking.values, 'med_volume': media_volume.values, 'std':std.values})
    
    # Concatene o dataframe temporário ao dataframe resultante
    df_resultante = pd.concat([df_resultante, df_temporario], ignore_index=True)

# Agrupe o dataframe resultante novamente por 'ativo' e calcule a média da coluna 'med_volume'
df_resultante = df_resultante.groupby('ativo').agg({'soma_ranking': 'sum', 'med_volume': 'mean', 'std':'sum'}).reset_index()


# PAPEIS FINAIS

In [93]:
#volume acima de 1MM
df_resultante = df_resultante.loc[df_resultante['med_volume'] >= 100000,:].sort_values(by='soma_ranking', ascending=False)

In [96]:
df_resultante.to_excel('compras.xlsx', index=False)
df_resultante.head(50)

,ativo,soma_ranking,med_volume,std
134,OIBR3,0.053877,7.797195e+05,0.009383
126,MRVE3,0.046336,4.852317e+07,0.029360
123,MODL3,0.041985,2.471325e+07,0.013773
143,PNVL3,0.039710,5.172274e+06,0.028019
91,HBSA3,0.034125,9.083056e+06,0.020839
135,ONCO3,0.033765,1.083729e+07,0.039727
41,CEAB3,0.028059,1.053093e+07,0.031956
144,POMO4,0.027406,2.159284e+07,0.015210
98,INTB3,0.024522,2.390210e+07,0.020472
119,MEGA3,0.024360,1.177323e+07,0.016136


In [87]:
#df_resultante.tail(50).to_excel('vendas.xlsx', index=False)